In [1]:
print("hello world")

hello world


In [2]:
import os 
os.chdir("../")

In [3]:
%cd fitness-ai-application

[WinError 2] The system cannot find the file specified: 'fitness-ai-application'
c:\Users\Gurnoor\Desktop\fitness ai\fitness-ai-application\research


In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [4]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf_file("data/")

In [7]:
#extracted_text

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("Lenght of text_chunks: ", len(text_chunks))

Lenght of text_chunks:  1763


In [7]:
#text_chunks

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
def download_hugging_face_embeddings():
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')
    return embedding

In [10]:
# Download the embeddings
embeddings = download_hugging_face_embeddings()

C:\Users\Gurnoor\AppData\Local\Temp\ipykernel_4412\1419061747.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')
c:\Users\Gurnoor\anaconda3\envs\fitbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query_results = embeddings.embed_query("Hello world")
print("Query results: ", len(query_results))

Query results:  384


In [12]:
from dotenv import load_dotenv
load_dotenv()


True

In [25]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPEANAI_API_KEY = os.environ.get("OPEANAI_API_KEY")


In [52]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key= PINECONE_API_KEY)

index_name = "fitbot"

pc.create_index(
    name=index_name,
    dimension= 384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [26]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPEANAI_API_KEY"] = OPEANAI_API_KEY

In [16]:
from langchain.vectorstores import Pinecone


In [ ]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings,
)

KeyboardInterrupt: 

In [17]:
#Load Existing Index

from langchain_pinecone import PineconeVectorStore 
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)

In [18]:
docsearch

In [19]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k": 3})

In [20]:
retrieved_documents = retriever.invoke("what is a muscle")

In [21]:
retrieved_documents

[Document(id='0bd7dd0a-17cd-41e5-8f94-c9ac8eda2c6f', metadata={'page': 183.0, 'source': "data\\Complete Idiot's Guide to Fitness.pdf"}, page_content='the classic \nmuscles that \nshows muscle \nmass'),
 Document(id='566d9b35-5ea0-4dd6-9039-9ae35b07bf7c', metadata={'page': 183.0, 'source': "data\\Complete Idiot's Guide to Fitness.pdf"}, page_content='the classic \nmuscles that \nshows muscle \nmass'),
 Document(id='b5310655-0151-4ada-844b-7e60a05659fa', metadata={'page': 183.0, 'source': "data\\Complete Idiot's Guide to Fitness.pdf"}, page_content='the classic \nmuscles that \nshows muscle \nmass')]

In [29]:
from langchain_openai import OpenAI

# Ensure the API key is set
os.environ["OPENAI_API_KEY"] = OPEANAI_API_KEY

llm = OpenAI(temperature=0.4, max_tokens=500)

In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a assistant for questions-answering tasks for a fitness app. "
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't have enough information, you can"
    "say you dont know. Use three sentences maximum and keep the answer concise."
    "if the user asks for information irrelevant to the context, you can say you dont know."
    "if the user asks for information about their health you may ask them more questions to give them a soultion"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [47]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [48]:
response = rag_chain.invoke({"input": "what should i eat to lose weight?"})
print(response['answer'])



You should focus on incorporating a balanced and nutritious diet that includes good fats, lean proteins, and plenty of fruits and vegetables. Avoid crash dieting or drastically reducing your food intake, as this can be counterproductive and harmful to your health. It is also important to incorporate regular exercise into your weight loss journey.
